# Azure Kusto
In this notebook we will access Azure Kusto database

In [8]:
# install all the azure kusto modules
%pip install azure.kusto.data
%pip install azure-kusto-ingest
%pip install azure.identity
%pip install pandas

from azure.kusto.data import KustoClient, KustoConnectionStringBuilder, ClientRequestProperties
from azure.identity import AzureCliCredential, ChainedTokenCredential, DefaultAzureCredential, ManagedIdentityCredential
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table
import pandas as pd
import json

class KustoConnection():
    def __init__(self, configuration : dict):
        uri = configuration.get('cluster_uri')
        if (not str(uri).startswith('https://')):
            uri = "https://" + uri
        if (not str(uri).endswith('.kusto.windows.net')) :
            uri = uri + '.kusto.windows.net'
        self.cluster_uri = uri
        self.database = configuration.get('database')
        cred = ChainedTokenCredential(ManagedIdentityCredential(), AzureCliCredential(), DefaultAzureCredential())
        connection_string = KustoConnectionStringBuilder.with_aad_application_token_authentication(self.cluster_uri, 
                                                                                                   cred.get_token(self.cluster_uri + '/.default').token)        
        # Create a KustoClient instance
        self.client = KustoClient(connection_string)

    def query(self, query: str, properties: ClientRequestProperties = None) -> pd.DataFrame:
        """
        Execute a Kusto query.
        @param query: The query.
        @returns: A Pandas DataFrame.
        """
        response = self.client.execute_query(self.database, query, properties)
        df = dataframe_from_result_table(response.primary_results[0])
        return df

    def get_schema(self, table_name: str) -> dict:
        """
        Get a table schema
        @param table_name: name of table
        @returns a dictionary mapping column names to kusto types
        """
        schema = self.client.execute_mgmt(self.database, f".show table {table_name} schema as json").primary_results[0][0]['Schema']
        return json.loads(schema)

    def close(self):
        """
        Close the Kusto client.
        """
        self.client.close()

class DCEPKustoData():
    def __init__(self):
        self.kusto_client = KustoConnection({'cluster_uri' : "dpxedwhprod.southcentralus", 'database': "Dpxedwhprod"});
        self.dcshortcode_selection = {
        'DC_List' : 
        '''
        let IFP_Done = 
        DCEPAllMilestonesCurrent
          | where ProjectType in ("Build","Build PH","OAI Build") 
          |summarize by ProjectName, Tranche
          | join kind = leftouter
          (
            DCEPAllMilestonesCurrent 
            | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by DCShortCode, 
                  ProjectName,Tranche, 
                  L2Milestone = "Design IFP", 
                  ProjectType, 
                  IFP_TaskStartDate = DesignIFP, 
                  IFP_TaskFinishDate = DesignIFP        
          )
          on $left.ProjectName == $right.ProjectName 
          | project-away ProjectName1, Tranche1
          | where IFP_TaskFinishDate <= ago(1d);
        let CR_NotDone = 
          DCEPAllMilestonesCurrent 
          | summarize by DCShortCode, Tranche, 
                L2Milestone = "Colo Ready", 
                ProjectType, 
                CR_TaskStartDate = ColoReadyDate, 
                CR_TaskFinishDate = ColoReadyDate 
          | where CR_TaskFinishDate > ago(2d); 
        IFP_Done 
        | join kind=inner(CR_NotDone) 
        on $left.ProjectType == $right.ProjectType and $left.DCShortCode == $right.DCShortCode and $left.Tranche == $right.Tranche
        | project DCShortCode, Tranche
        | order by DCShortCode, Tranche asc
        ''',
        
        'IST2CR_List' : 
        '''
        let IFPFullSet = 
        (
            DCEPAllMilestonesCurrent
            | where ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by ProjectName, Tranche
            | join kind = leftouter
            (
              DCEPAllMilestonesCurrent 
              | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
              | summarize by DCShortCode, 
                    ProjectName,Tranche, 
                    L2Milestone = "Design IFP", 
                    ProjectType, 
                    IFP_TaskStartDate = DesignIFP, 
                    IFP_TaskFinishDate = DesignIFP          
            )
            on $left.ProjectName == $right.ProjectName 
            | project-away ProjectName1, Tranche1
        ); 
        let CSFullSet = 
        (
            DCEPAllMilestonesCurrent
            | where ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by ProjectName, Tranche
            | join kind = leftouter
            (
              DCEPAllMilestonesCurrent 
              | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
              | summarize by DCShortCode, ProjectName,
                    Tranche, L2Milestone = "Construction Start", 
                    ProjectType, CS_TaskStartDate = ConstructionDate, 
                    CS_TaskFinishDate = ConstructionDate            
            )
            on $left.ProjectName == $right.ProjectName 
            | project-away ProjectName1, Tranche1
        ); 
        let FSFullSet = 
        ( 
            EstimatedFoundationStart
            | summarize by 
                  DCShortCode, ProjectName, Tranche = rostring(Tranche), L2Milestone="Foundation", ProjectType, 
                  FS_TaskStartDate = Foudation_TaskStartDate, 
                  FS_TaskFinishDate = Foudation_TaskFinishDate
        ); 
        let ISTFullSet = 
        (
            DCEPAllMilestonesCurrent 
            | where ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by DCShortCode, Tranche, 
                  L2Milestone = "IST Complete", ProjectType, 
                  IST_TaskStartDate = ISTDate, IST_TaskFinishDate = ISTDate
        ); 
        let CRFullSet =  
        (
            DCEPAllMilestonesCurrent 
            | where ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by DCShortCode, Tranche, 
                  L2Milestone = "Colo Ready", ProjectType, 
                  CR_TaskStartDate = ColoReadyDate, 
                  CR_TaskFinishDate = ColoReadyDate
        ); 
        IFPFullSet 
          | join kind= inner (CSFullSet) 
          on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType and $left.Tranche == $right.Tranche 
          | join kind= inner (FSFullSet) 
          on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType and $left.Tranche == $right.Tranche 
          | join kind= inner (ISTFullSet) 
          on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType and $left.Tranche == $right.Tranche 
          | join kind= inner (CRFullSet) 
          on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType and $left.Tranche == $right.Tranche 
          | where IFP_TaskFinishDate <= ago(1d) and CS_TaskStartDate  <= ago(1d) 
             and FS_TaskStartDate <= ago(1d) and IST_TaskStartDate <= ago(1d) 
             and CR_TaskStartDate > ago(1d) 
          | summarize by DCShortCode, Tranche
        ''',
        
        'T2_FS2CR_List' : 
        '''
        let IFPFullSet = 
        (
            DCEPAllMilestonesCurrent 
            | where ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by ProjectName, Tranche
            | join kind = leftouter  
            (
              DCEPAllMilestonesCurrent 
              | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
              | summarize by DCShortCode, ProjectName,Tranche, 
                    L2Milestone = "Design IFP", ProjectType, IFP_TaskStartDate = DesignIFP, 
                    IFP_TaskFinishDate = DesignIFP
            )
            on $left.ProjectName == $right.ProjectName 
            | project-away ProjectName1, Tranche1
        ); 
        let CSFullSet = 
        (
            DCEPAllMilestonesCurrent
            | where ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by ProjectName, Tranche
            | join kind = leftouter  
            (
                DCEPAllMilestonesCurrent 
                | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
                | summarize by DCShortCode, ProjectName,Tranche, 
                      L2Milestone = "Construction Start", ProjectType, 
                      CS_TaskStartDate = ConstructionDate, CS_TaskFinishDate = ConstructionDate
            )
            on $left.ProjectName == $right.ProjectName 
            | project-away ProjectName1, Tranche1
        ); 
        let FSFullSet = 
        ( 
          EstimatedFoundationStart
          | where ProjectType in ("Build","Build PH","OAI Build") 
          | summarize by DCShortCode, ProjectName,Tranche = tostring(Tranche), 
                L2Milestone="Foundation", ProjectType, 
                FS_TaskStartDate = Foudation_TaskStartDate, 
                FS_TaskFinishDate = Foudation_TaskFinishDate
        ); 
        let ISTFullSet = 
        (
          DCEPAllMilestonesCurrent 
          | where ProjectType in ("Build","Build PH","OAI Build") 
          | summarize by DCShortCode, Tranche, L2Milestone = "IST Complete", 
                ProjectType, IST_TaskStartDate = ISTDate, IST_TaskFinishDate = ISTDate
        ); 
        let CRFullSet = 
        (
          DCEPAllMilestonesCurrent 
          | where ProjectType in ("Build","Build PH","OAI Build") 
          | summarize by DCShortCode, Tranche, L2Milestone = "Colo Ready", 
                ProjectType, CR_TaskStartDate = ColoReadyDate, 
                CR_TaskFinishDate = ColoReadyDate
        ); 
        IFPFullSet 
          | join kind= inner (CSFullSet) 
            on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType and $left.Tranche == $right.Tranche
          | join kind= inner (FSFullSet) 
            on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType and $left.Tranche == $right.Tranche
          | join kind= inner (ISTFullSet) 
            on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType and $left.Tranche == $right.Tranche
          | join kind= inner (CRFullSet) 
            on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType and $left.Tranche == $right.Tranche
          | where IFP_TaskFinishDate <= ago(1d) and CS_TaskStartDate  <= ago(1d) 
               and FS_TaskStartDate <= ago(1d) and IST_TaskStartDate > ago(1d) 
               and CR_TaskStartDate > ago(1d) 
          | summarize by DCShortCode, Tranche
        ''',
        
        'T2_CS2CR_List' :
        ''' 
        let IFPFullSet = 
        (
            DCEPAllMilestonesCurrent
            | where ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by ProjectName, Tranche
            | join kind = leftouter  
            (
                DCEPAllMilestonesCurrent 
                | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
                | summarize by DCShortCode, ProjectName,Tranche, L2Milestone = "Design IFP", 
                  ProjectType, IFP_TaskStartDate = DesignIFP, IFP_TaskFinishDate = DesignIFP
            ) on $left.ProjectName == $right.ProjectName 
            | project-away ProjectName1, Tranche1 
        ); 
        let CSFullSet = 
        (
            DCEPAllMilestonesCurrent
            | where ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by ProjectName, Tranche
            | join kind = leftouter  
            (
                DCEPAllMilestonesCurrent 
                | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
                | summarize by DCShortCode, ProjectName,Tranche, 
                  L2Milestone = "Construction Start", ProjectType, 
                  CS_TaskStartDate = ConstructionDate, CS_TaskFinishDate = ConstructionDate
            ) on $left.ProjectName == $right.ProjectName 
            | project-away ProjectName1, Tranche1
        ); 
        let FSFullSet = 
        ( 
            EstimatedFoundationStart
            | where ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by 
                  DCShortCode, ProjectName,Tranche = to_string(Tranche), 
                  L2Milestone="Foundation", 
                  ProjectType, FS_TaskStartDate = Foudation_TaskStartDate, 
                  FS_TaskFinishDate = Foudation_TaskFinishDate
        ); 
        let ISTFullSet = 
        (
            DCEPAllMilestonesCurrent 
            | where ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by 
                DCShortCode, Tranche, L2Milestone = "IST Complete", 
                ProjectType, IST_TaskStartDate = ISTDate, IST_TaskFinishDate = ISTDate
        ); 
        let CRFullSet =  
        (
            DCEPAllMilestonesCurrent 
            | where ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by DCShortCode, Tranche, L2Milestone = "Colo Ready", 
              ProjectType, CR_TaskStartDate = ColoReadyDate, CR_TaskFinishDate = ColoReadyDate
        ); 
        IFPFullSet 
        | join kind= inner (CSFullSet) 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
        | join kind= inner (FSFullSet) 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
        | join kind= inner (ISTFullSet) 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType
        | join kind= inner (CRFullSet) 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
        | where IFP_TaskFinishDate <= ago(1d) and CS_TaskStartDate  <= ago(1d) 
          and FS_TaskStartDate  > ago(1d) and IST_TaskStartDate > ago(1d) 
          and CR_TaskStartDate > ago(1d) 
        | summarize by DCShortCode, Tranche
        ''',
        
        'T2_IFP2CR_List' :
        '''
        let IFPFullSet = 
        (
            DCEPAllMilestonesCurrent
            | where ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by ProjectName, Tranche| join kind = leftouter  
            (
                DCEPAllMilestonesCurrent 
                | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
                | summarize by DCShortCode, ProjectName,Tranche, L2Milestone = "Design IFP", 
                  ProjectType, IFP_TaskStartDate = DesignIFP, IFP_TaskFinishDate = DesignIFP
            )
            on $left.ProjectName == $right.ProjectName 
            | project-away ProjectName1, Tranche1
            | where Tranche ==2
        ); 
        let CSFullSet = 
        (
            DCEPAllMilestonesCurrent
            | where ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by ProjectName, Tranche
            | join kind = leftouter  
            (
                DCEPAllMilestonesCurrent 
                | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
                | summarize by DCShortCode, ProjectName,Tranche, 
                  L2Milestone = "Construction Start", ProjectType, 
                  CS_TaskStartDate = ConstructionDate, CS_TaskFinishDate = ConstructionDate
            )
            on $left.ProjectName == $right.ProjectName 
            | project-away ProjectName1, Tranche1
            | where Tranche ==2
        ); 
        let FSFullSet = 
        ( 
            EstimatedFoundationStart
            | where Tranche ==2
            | summarize by DCShortCode, ProjectName,Tranche, 
              L2Milestone="Foundation", ProjectType, 
              FS_TaskStartDate = Foudation_TaskStartDate, 
              FS_TaskFinishDate = Foudation_TaskFinishDate
        ); 
        let ISTFullSet = 
        (
            DCEPAllMilestonesCurrent 
            | where Tranche == 2 and ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by DCShortCode, Tranche, L2Milestone = "IST Complete", 
              ProjectType, IST_TaskStartDate = ISTDate, IST_TaskFinishDate = ISTDate
        ); 
        let CRFullSet=  
        (
            DCEPAllMilestonesCurrent 
            | where Tranche == 2 and ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by DCShortCode, Tranche, L2Milestone = "Colo Ready", 
              ProjectType, CR_TaskStartDate = ColoReadyDate, CR_TaskFinishDate = ColoReadyDate
        ); 
        IFPFullSet 
        | join kind= inner (CSFullSet) 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
        | join kind= inner (FSFullSet) 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
        | join kind= inner (ISTFullSet) 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType
        | join kind= inner (CRFullSet) 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
        | where IFP_TaskFinishDate <= ago(1d) and CS_TaskStartDate  > ago(1d) 
          and FS_TaskStartDate  > ago(1d) and IST_TaskStartDate > ago(1d) 
          and CR_TaskStartDate > ago(1d) 
        | summarize by DCShortCode
        ''' 
        };
    
        self.duration = {
        'IST_CR' : 
        '''
        let ISTFullSet = 
        (
            DCEPAllMilestonesCurrent 
            | where ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by 
                  DCShortCode, Tranche, L2Milestone = "IST Complete", ProjectType, 
                  IST_TaskStartDate = ISTDate, IST_TaskFinishDate = ISTDate
        ); 
        let CRFullSet =  
        (
            DCEPAllMilestonesCurrent
            | where ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by 
                  DCShortCode, Tranche, L2Milestone = "Colo Ready", ProjectType, 
                  CR_TaskStartDate = ColoReadyDate, CR_TaskFinishDate = ColoReadyDate
        ); 
        ISTFullSet 
        | join kind= inner (CRFullSet) 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
        | extend duration = datetime_diff("day",CR_TaskFinishDate, IST_TaskFinishDate)
        ''',
        
        'FS_CR' : 
        '''
        let FSFullSet = 
        (
            EstimatedFoundationStart
            | where ProjectType in ("Build","Build PH","OAI Build")
            | summarize by 
                  DCShortCode, Tranche = to_string(Tranche), 
                  L2Milestone="Foundation", ProjectType, 
                  FS_TaskStartDate = Foudation_TaskStartDate, 
                  FS_TaskFinishDate = Foudation_TaskFinishDate
        ); 
        let CRFullSet = 
        (
          DCEPAllMilestonesCurrent 
          | where ProjectType in ("Build","Build PH","OAI Build") 
            and ProjectType in ("Buil 
          | summarize by DCShortCode, Tranche, L2Milestone = "Colo Ready", ProjectType, CR_TaskStartDate = ColoReadyDate, CR_TaskFinishDate = ColoReadyDate
        ); 
        FSFullSet 
        | join kind= inner (CRFullSet) 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
        | extend duration = datetime_diff("day", CR_TaskFinishDate, FS_TaskStartDate)
        ''',
        
        'T2_CS_CR' : 
        '''
        let CSFullSet = 
        (
          DCEPAllMilestonesCurrent
          | where ProjectType in ("Build","Build PH","OAI Build") 
          | summarize by ProjectName, Tranche
          | join kind = leftouter  
          (
            DCEPAllMilestonesCurrent 
            | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build")
            | summarize by DCShortCode, ProjectName,Tranche, L2Milestone = "Construction Start", 
                           ProjectType, CS_TaskStartDate = ConstructionDate, 
                           CS_TaskFinishDate = ConstructionDate
          ) 
          on $left.ProjectName == $right.ProjectName 
          | project-away ProjectName1, Tranche1
          | where Tranche ==2 and isnotempty(CS_TaskFinishDate)
        ); 
        let CRFullSet = 
        (
          DCEPAllMilestonesCurrent 
          | where Tranche == 2 and ProjectType in ("Build","Build PH","OAI Build") and DCShortCode == "CHI05" 
          | summarize by DCShortCode, Tranche, L2Milestone = "Colo Ready", ProjectType, 
                         CR_TaskStartDate = ColoReadyDate, CR_TaskFinishDate = ColoReadyDate
        ); 
        CSFullSet 
        | join kind = inner (CRFullSet) 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
        | extend duration = datetime_diff("day", CR_TaskFinishDate, CS_TaskFinishDate)
        ''',
        
        'T2_IFP_CR' :
        ''' 
        let IFPFullSet = 
        (
          DCEPAllMilestonesCurrent
          | where ProjectType in ("Build","Build PH","OAI Build") 
          | summarize by ProjectName, Tranche
          | join kind = leftouter  
          (
            DCEPAllMilestonesCurrent 
            | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by DCShortCode, ProjectName,Tranche, L2Milestone = "Design IFP", ProjectType, 
                           IFP_TaskStartDate = DesignIFP, IFP_TaskFinishDate = DesignIFP
          )
          on $left.ProjectName == $right.ProjectName 
          | project-away ProjectName1, Tranche1
          | where Tranche ==2
        ); 
        let CRFullSet = 
        (
          DCEPAllMilestonesCurrent 
          | where Tranche == 2 and ProjectType in ("Build","Build PH","OAI Build") and DCShortCode == "CHI05" 
          | summarize by DCShortCode, Tranche, L2Milestone = "Colo Ready", ProjectType, 
                         CR_TaskStartDate = ColoReadyDate, CR_TaskFinishDate = ColoReadyDate
        ); 
        IFPFullSet 
        | join kind = inner (CRFullSet) 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
        | extend duration = datetime_diff("day", CR_TaskFinishDate, IFP_TaskFinishDate)
        ''',
        };
     
        self.statistics = {
        'T2_IFP_CR_p5080' : 
        '''
        let FY = 
        iff(
            monthofyear(ago(0d)) > 6, 
            datetime_part("year", ago(0d)) + 1, 
            datetime_part("year", ago(0d))
            ); 
        let FY_Startdate = strcat(FY - 3, "-07-01"); 
        let FY_Enddate = strcat(FY - 1, "-06-30"); 
        let CRFullSet = DCEPAllMilestonesCurrent 
        | where ProjectStatus == "Complete" and Tranche == 2 
        and ProjectType in ("Build","Build PH","OAI Build") 
        and ColoReadyDate <= todatetime(FY_Enddate) 
        and ColoReadyDate > todatetime(FY_Startdate) 
        | summarize minif(ColoReadyDate, Tranche == 2) 
          by DCShortCode, ProjectType, ProjectName, Tranche, Region, ColoReadyDate; 
        let IFPFullSet =  DCEPAllMilestonesCurrent
        | where ProjectType in ("Build","Build PH","OAI Build") 
        | summarize by ProjectName, Tranche
        | join kind = leftouter  
        (
            DCEPAllMilestonesCurrent 
            | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by DCShortCode, ProjectName,Tranche, Region, ProjectType, 
              IFP_TaskStartDate = DesignIFP, IFP_TaskFinishDate = DesignIFP
        )
        on $left.ProjectName == $right.ProjectName 
        | project-away ProjectName1, Tranche1
        | where Tranche ==2 and isnotempty(IFP_TaskStartDate); 
        let IFP_CR_Duration = IFPFullSet 
        | join kind = inner CRFullSet on $left.DCShortCode == $right.DCShortCode 
          and $left.ProjectType == $right.ProjectType 
        | project DCShortCode, IFP_TaskStartDate, ColoReadyDate, 
          duration = datetime_diff("day",ColoReadyDate, IFP_TaskStartDate) 
        | extend leadtime = datetime_diff("day",ColoReadyDate, IFP_TaskStartDate)*1.0 
        | order by DCShortCode asc 
        | extend ID="IFPCR"
        | where duration  > 0; 
        let IFP_CR_Outlier = IFP_CR_Duration 
        | summarize P75 = percentiles(leadtime,75),P25 = percentiles(leadtime,25) 
        | extend IQR = P75-P25 
        | extend Upper_Whisker = P75+1.5*IQR ,Lower_Whisker = P25-1.5*IQR 
        | extend ID="IFPCR"; let IFP_CR = IFP_CR_Duration 
        | join kind = leftouter IFP_CR_Outlier on $left.ID == $right.ID 
        | extend Outlier = 
        iff(leadtime >Upper_Whisker or leadtime < Lower_Whisker,"Outlier","Not Outlier") 
        | where Outlier != "Outlier"; IFP_CR 
        | summarize percentiles(duration, 50,80)
        ''',
        
        'T2_IFP_CR_minmax' : 
        '''
        let FY = iff
        ( 
            monthofyear(ago(0d)) > 6, 
            datetime_part("year", ago(0d)) + 1, 
            datetime_part("year", ago(0d))
        ); 
        let FY_Startdate = strcat(FY - 3, "-07-01"); 
        let FY_Enddate = strcat(FY - 1, "-06-30"); 
        let CRFullSet = 
        DCEPAllMilestonesCurrent 
        | where ProjectStatus == "Complete" and Tranche == 2 
          and ProjectType in ("Build","Build PH","OAI Build") 
          and ColoReadyDate <= todatetime(FY_Enddate) 
          and ColoReadyDate > todatetime(FY_Startdate) 
        | summarize minif(ColoReadyDate, Tranche == 2) 
          by DCShortCode, ProjectType, ProjectName, Tranche, Region, ColoReadyDate; 
        let IFPFullSet =  
        DCEPAllMilestonesCurrent
        | where ProjectType in ("Build","Build PH","OAI Build") 
        | summarize by ProjectName, Tranche 
        | join kind = leftouter  
        (
            DCEPAllMilestonesCurrent 
            | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by DCShortCode, ProjectName,Tranche, Region, ProjectType, 
              IFP_TaskStartDate = DesignIFP, IFP_TaskFinishDate = DesignIFP
        )
        on $left.ProjectName == $right.ProjectName 
        | project-away ProjectName1, Tranche1
        | where Tranche ==2 and isnotempty(IFP_TaskStartDate); 
        let IFP_CR_Duration = IFPFullSet 
        | join kind = inner CRFullSet 
        on $left.DCShortCode == $right.DCShortCode 
        and $left.ProjectType == $right.ProjectType 
        | project DCShortCode, IFP_TaskStartDate, ColoReadyDate, 
          duration = datetime_diff("day",ColoReadyDate, IFP_TaskStartDate) 
        | extend leadtime = datetime_diff("day",ColoReadyDate, IFP_TaskStartDate)*1.0 
        | order by DCShortCode asc | extend ID="IFPCR"
        | where duration  > 0; 
        let IFP_CR_Outlier = IFP_CR_Duration 
        | summarize P75 = percentiles(leadtime,75),P25 = percentiles(leadtime,25) 
        | extend IQR = P75-P25 
        | extend Upper_Whisker = P75+1.5*IQR ,Lower_Whisker = P25-1.5*IQR 
        | extend ID="IFPCR"; let IFP_CR = IFP_CR_Duration 
        | join kind = leftouter IFP_CR_Outlier on $left.ID == $right.ID 
        | extend Outlier = 
          iff(leadtime >Upper_Whisker or leadtime < Lower_Whisker,"Outlier","Not Outlier") 
        | where Outlier != "Outlier"; 
        IFP_CR 
        | summarize min(duration), max(duration)
        ''',
        
        'T2_CS_CR_p5080' : 
        '''
        let FY = iff(
            monthofyear(ago(0d)) > 6, 
            datetime_part("year", ago(0d)) + 1, 
            datetime_part("year", ago(0d))
        ); 
        let FY_Startdate = strcat(FY - 3, "-07-01"); 
        let FY_Enddate = strcat(FY - 1, "-06-30"); 
        let CRFullSet = DCEPAllMilestonesCurrent 
        | where ProjectStatus == "Complete" and Tranche == 2 
          and ProjectType in ("Build","Build PH","OAI Build") 
          and ColoReadyDate <= todatetime(FY_Enddate) 
          and ColoReadyDate > todatetime(FY_Startdate) 
        | summarize minif(ColoReadyDate, Tranche == 2) 
          by DCShortCode, ProjectType, ProjectName, Tranche, Region, ColoReadyDate;
        let CSFullSet = DCEPAllMilestonesCurrent
        | where ProjectType in ("Build","Build PH","OAI Build") 
        | summarize by ProjectName, Tranche
        | join kind = leftouter 
        (
            DCEPAllMilestonesCurrent 
            | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by DCShortCode, ProjectName,Tranche, Region,
              L2Milestone = "Construction Start", ProjectType, 
              CS_TaskStartDate = ConstructionDate, 
              CS_TaskFinishDate = ConstructionDate
        )
        on $left.ProjectName == $right.ProjectName 
        | project-away ProjectName1, Tranche1
        | where Tranche ==2 and isnotempty(CS_TaskStartDate)
        | summarize CS_TaskStartDate = minif(CS_TaskStartDate, 
           L2Milestone in ("EarthWorks", "Construction Start")) by 
           DCShortCode, ProjectType, ProjectName, Tranche, Region; 
        let CS_CR_Duration = CSFullSet 
        | join kind = inner CRFullSet 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
        | project DCShortCode, CS_TaskStartDate, ColoReadyDate, 
          duration = datetime_diff("day",ColoReadyDate, CS_TaskStartDate) 
        | extend leadtime = datetime_diff("day",ColoReadyDate, CS_TaskStartDate)*1.0 
        | order by DCShortCode asc | extend ID="CSCR"| where duration  > 0; 
        let CS_CR_Outlier = CS_CR_Duration 
        | summarize P75 = percentiles(leadtime,75), 
          P25 = percentiles(leadtime,25) 
        | extend IQR = P75-P25 
        | extend Upper_Whisker = P75+1.5*IQR ,Lower_Whisker = P25-1.5*IQR 
        | extend ID="CSCR"; let CS_CR = CS_CR_Duration 
        | join kind = leftouter CS_CR_Outlier on $left.ID == $right.ID 
        | extend Outlier = iff(
            leadtime >Upper_Whisker or leadtime < Lower_Whisker,"Outlier","Not Outlier"
            ) 
        | where Outlier != "Outlier";CS_CR 
        | summarize percentiles(duration, 50,80)        
        ''',
        
        'T2_CS_CR_minmax' :
        ''' 
        let FY = iff(
            monthofyear(ago(0d)) > 6, 
            datetime_part("year", ago(0d)) + 1, 
            datetime_part("year", ago(0d))
            ); 
        let FY_Startdate = strcat(FY - 3, "-07-01"); 
        let FY_Enddate = strcat(FY - 1, "-06-30"); 
        let CRFullSet = DCEPAllMilestonesCurrent 
        | where ProjectStatus == "Complete" and Tranche == 2 
           and ProjectType in ("Build","Build PH","OAI Build") 
           and ColoReadyDate <= todatetime(FY_Enddate) 
           and ColoReadyDate > todatetime(FY_Startdate) 
        | summarize minif(ColoReadyDate, Tranche == 2) by 
           DCShortCode, ProjectType, ProjectName, Tranche, Region, 
           ColoReadyDate;let CSFullSet = DCEPAllMilestonesCurrent
        | where ProjectType in ("Build","Build PH","OAI Build") 
        | summarize by ProjectName, Tranche
        | join kind = leftouter   
        (
            DCEPAllMilestonesCurrent 
            | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by DCShortCode, ProjectName,Tranche, Region,
            L2Milestone = "Construction Start", ProjectType, 
            CS_TaskStartDate = ConstructionDate, 
            CS_TaskFinishDate = ConstructionDate
        )
        on $left.ProjectName == $right.ProjectName 
        | project-away ProjectName1, Tranche1
        | where Tranche ==2 and isnotempty(CS_TaskStartDate) 
        | summarize CS_TaskStartDate = minif(
            CS_TaskStartDate, L2Milestone in ("EarthWorks", "Construction Start")
            ) by DCShortCode, ProjectType, ProjectName, Tranche, 
            Region; 
        let CS_CR_Duration = CSFullSet 
        | join kind = inner CRFullSet on $left.DCShortCode == $right.DCShortCode 
          and $left.ProjectType == $right.ProjectType 
        | project DCShortCode, CS_TaskStartDate, ColoReadyDate, 
          duration = datetime_diff("day",ColoReadyDate, CS_TaskStartDate) 
        | extend leadtime = datetime_diff("day",ColoReadyDate, CS_TaskStartDate)*1.0 
        | order by DCShortCode asc 
        | extend ID="CSCR"
        | where duration  > 0; 
        let CS_CR_Outlier = CS_CR_Duration 
        | summarize P75 = percentiles(leadtime,75),
           P25 = percentiles(leadtime,25) 
        | extend IQR = P75-P25 
        | extend Upper_Whisker = P75+1.5*IQR ,Lower_Whisker = P25-1.5*IQR 
        | extend ID="CSCR"; let CS_CR = CS_CR_Duration 
        | join kind = leftouter CS_CR_Outlier on $left.ID == $right.ID 
        | extend Outlier = iff(
            leadtime >Upper_Whisker or leadtime < Lower_Whisker,
            "Outlier","Not Outlier"
            ) 
        | where Outlier != "Outlier";CS_CR 
        | summarize min(duration), max(duration)
        ''',
        
        'T2_FS_CR_p5080' :
        '''
        let FY = iff(
            monthofyear(ago(0d)) > 6, 
            datetime_part("year", ago(0d)) + 1, 
            datetime_part("year", ago(0d))
            ); 
        let FY_Startdate = strcat(FY - 3, "-07-01"); 
        let FY_Enddate = strcat(FY - 1, "-06-30"); 
        let CRFullSet = DCEPAllMilestonesCurrent 
        | where ProjectStatus == "Complete" and Tranche == 2 
          and ProjectType in ("Build","Build PH","OAI Build") 
          and ColoReadyDate <= todatetime(FY_Enddate) 
          and ColoReadyDate > todatetime(FY_Startdate) 
        | summarize minif(ColoReadyDate, Tranche == 2) 
        by DCShortCode, ProjectType, ProjectName, Tranche, Region, ColoReadyDate; 
        let FSFullSet =EstimatedFoundationStart
        | where Tranche ==2
        | summarize by DCShortCode, ProjectType, ProjectName, 
          Tranche,Region,FS_TaskStartDate =Foudation_TaskStartDate; 
        let FS_CR_Duration = FSFullSet 
        | join kind = inner CRFullSet 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
        | project DCShortCode, FS_TaskStartDate, ColoReadyDate, 
          duration = datetime_diff("day",ColoReadyDate, FS_TaskStartDate) 
        | extend leadtime = datetime_diff(
            "day",ColoReadyDate, FS_TaskStartDate)*1.0 
        | order by DCShortCode asc 
        | extend ID="FSCR"
        | where duration  > 0; 
        let FS_CR_Outlier = FS_CR_Duration 
        | summarize P75 = percentiles(leadtime,75),P25 = percentiles(leadtime,25) 
        | extend IQR = P75-P25 
        | extend Upper_Whisker = P75+1.5*IQR ,Lower_Whisker = P25-1.5*IQR 
        | extend ID="FSCR"; 
        let FS_CR = FS_CR_Duration 
        | join kind = leftouter FS_CR_Outlier on $left.ID == $right.ID 
        | extend Outlier = iff(
            leadtime >Upper_Whisker or leadtime < Lower_Whisker,"Outlier","Not Outlier"
            ) 
        | where Outlier != "Outlier"; 
        FS_CR | summarize percentiles(duration, 50,80)
        ''',
        'T2_FS_CR_minmax' :
        '''
        let FY = iff(
            monthofyear(ago(0d)) > 6, 
            datetime_part("year", ago(0d)) + 1, 
            datetime_part("year", ago(0d))
            ); 
        let FY_Startdate = strcat(FY - 3, "-07-01"); 
        let FY_Enddate = strcat(FY - 1, "-06-30"); 
        let CRFullSet = DCEPAllMilestonesCurrent 
        | where ProjectStatus == "Complete" and Tranche == 2 
          and ProjectType in ("Build","Build PH","OAI Build") 
          and ColoReadyDate <= todatetime(FY_Enddate) 
          and ColoReadyDate > todatetime(FY_Startdate) 
        | summarize minif(ColoReadyDate, Tranche == 2) 
          by DCShortCode, ProjectType, ProjectName, Tranche, 
             Region, ColoReadyDate; let FSFullSet =EstimatedFoundationStart
        | where Tranche ==2
        | summarize by DCShortCode, ProjectType, ProjectName, Tranche,Region,
          FS_TaskStartDate =Foudation_TaskStartDate; 
        let FS_CR_Duration = FSFullSet 
        | join kind = inner CRFullSet 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
        | project DCShortCode, FS_TaskStartDate, ColoReadyDate, 
          duration = datetime_diff("day",ColoReadyDate, FS_TaskStartDate) 
        | extend leadtime = datetime_diff("day",ColoReadyDate, FS_TaskStartDate)*1.0 
        | order by DCShortCode asc 
        | extend ID="FSCR"
        | where duration  > 0; let FS_CR_Outlier = FS_CR_Duration 
        | summarize P75 = percentiles(leadtime,75),P25 = percentiles(leadtime,25) 
        | extend IQR = P75-P25 
        | extend Upper_Whisker = P75+1.5*IQR ,Lower_Whisker = P25-1.5*IQR 
        | extend ID="FSCR"; let FS_CR = FS_CR_Duration 
        | join kind = leftouter FS_CR_Outlier on $left.ID == $right.ID 
        | extend Outlier = iff(
            leadtime >Upper_Whisker or leadtime < Lower_Whisker,"Outlier","Not Outlier"
            ) 
        | where Outlier != "Outlier"; 
        FS_CR | summarize min(duration), max(duration)
        ''',
        'T2_IST_CR_p5080' :
        '''
        let FY = iff(
            monthofyear(ago(0d)) > 6, 
            datetime_part("year", ago(0d)) + 1, 
            datetime_part("year", ago(0d))
        ); 
        let FY_Startdate = strcat(FY - 3, "-07-01"); 
        let FY_Enddate = strcat(FY - 1, "-06-30"); 
        let CRFullSet = DCEPAllMilestonesCurrent 
        | where ProjectStatus == "Complete" and Tranche == 2 
          and ProjectType in ("Build","Build PH","OAI Build") 
          and ColoReadyDate <= todatetime(FY_Enddate) 
          and ColoReadyDate > todatetime(FY_Startdate) 
        | summarize minif(ColoReadyDate, Tranche == 2) by 
          DCShortCode, ProjectType, ProjectName, Tranche, Region, ColoReadyDate; 
        let ISTFullSet = DCEPAllMilestonesCurrent 
        | where ProjectStatus == "Complete" and Tranche == 2 
          and ProjectType in ("Build","Build PH","OAI Build") 
          and ColoReadyDate <= todatetime(FY_Enddate) 
          and ColoReadyDate > todatetime(FY_Startdate) 
        | summarize minif(ColoReadyDate, Tranche == 2) by 
          DCShortCode, ProjectType, ProjectName, Tranche, Region, ISTDate; 
        let IST_CR_Duration = ISTFullSet 
        | join kind = inner CRFullSet 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
        | project DCShortCode, ISTDate, ColoReadyDate, 
        duration = datetime_diff("day",ColoReadyDate, ISTDate) 
        | extend leadtime = datetime_diff("day",ColoReadyDate, ISTDate)*1.0 
        | order by DCShortCode asc | extend ID="ISTCR"
        | where duration  > 0; 
        let IST_CR_Outlier = IST_CR_Duration 
        | summarize P75 = percentiles(leadtime,75),P25 = percentiles(leadtime,25) 
        | extend IQR = P75-P25 
        | extend Upper_Whisker = P75+1.5*IQR ,Lower_Whisker = P25-1.5*IQR 
        | extend ID="ISTCR"; let IST_CR = IST_CR_Duration 
        | join kind = leftouter IST_CR_Outlier on $left.ID == $right.ID 
        | extend Outlier = iff(
            leadtime >Upper_Whisker or leadtime < Lower_Whisker,"Outlier","Not Outlier"
            ) 
        | where Outlier != "Outlier"; 
        IST_CR | summarize percentiles(duration, 50,80)
        ''',
        'T2_IST_CR_minmax' :
        '''
        let FY = iff(
            monthofyear(ago(0d)) > 6, 
            datetime_part("year", ago(0d)) + 1, 
            datetime_part("year", ago(0d))
        ); 
        let FY_Startdate = strcat(FY - 3, "-07-01"); 
        let FY_Enddate = strcat(FY - 1, "-06-30"); 
        let CRFullSet = DCEPAllMilestonesCurrent 
        | where ProjectStatus == "Complete" and Tranche == 2 
          and ProjectType in ("Build","Build PH","OAI Build") 
          and ColoReadyDate <= todatetime(FY_Enddate) 
          and ColoReadyDate > todatetime(FY_Startdate) 
        | summarize minif(ColoReadyDate, Tranche == 2) by 
          DCShortCode, ProjectType, ProjectName, Tranche, Region, ColoReadyDate; 
        let ISTFullSet = DCEPAllMilestonesCurrent 
        | where ProjectStatus == "Complete" and Tranche == 2 
          and ProjectType in ("Build","Build PH","OAI Build") 
          and ColoReadyDate <= todatetime(FY_Enddate) 
          and ColoReadyDate > todatetime(FY_Startdate) 
        | summarize minif(ColoReadyDate, Tranche == 2) by 
          DCShortCode, ProjectType, ProjectName, Tranche, Region, ISTDate; 
        let IST_CR_Duration = ISTFullSet 
        | join kind = inner CRFullSet 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
        | project DCShortCode, ISTDate, ColoReadyDate, 
          duration = datetime_diff("day",ColoReadyDate, ISTDate) 
        | extend leadtime = datetime_diff("day",ColoReadyDate, ISTDate)*1.0 
        | order by DCShortCode asc | extend ID="ISTCR"
        | where duration  > 0; 
        let IST_CR_Outlier = IST_CR_Duration 
        | summarize P75 = percentiles(leadtime,75),P25 = percentiles(leadtime,25) 
        | extend IQR = P75-P25 
        | extend Upper_Whisker = P75+1.5*IQR ,Lower_Whisker = P25-1.5*IQR 
        | extend ID="ISTCR"; 
        let IST_CR = IST_CR_Duration 
        | join kind = leftouter IST_CR_Outlier on $left.ID == $right.ID 
        | extend Outlier = iff(
            leadtime >Upper_Whisker or leadtime < Lower_Whisker,"Outlier","Not Outlier"
            ) 
        | where Outlier != "Outlier"; 
        IST_CR | summarize min(duration), max(duration)
        '''
        };

    def get_duration_query(self):
        return self.duration;

    def query_kusto(self, query): 
        response = self.kusto_client.query(query);
        result = response.reset_index();         
        return result;

    def query_static_data(self):
        self.data = {};
        query = self.dcshortcode_selection['T2_IST2CR_List'];
        response = self.kusto_client.query(query);
        result = response.reset_index();
        self.data['T2_IST2CR_List'] = result;

        query = self.dcshortcode_selection['T2_FS2CR_List'];
        response = self.kusto_client.query(query);
        result = response.reset_index();
        self.data['T2_FS2CR_List'] = result;

        query = self.dcshortcode_selection['T2_CS2CR_List'];
        response = self.kusto_client.query(query);
        result = response.reset_index();
        self.data['T2_CS2CR_List'] = result;

        query = self.dcshortcode_selection['T2_IFP2CR_List'];
        response = self.kusto_client.query(query);
        result = response.reset_index();
        self.data['T2_IFP2CR_List'] = result;
        
        query = self.dcshortcode_selection['T2_DC_List'];
        response = self.kusto_client.query(query);
        result = response.reset_index();
        self.data['T2_DC_List'] = result;
        
        query = self.statistics['T2_IFP_CR_p5080'];
        response = self.kusto_client.query(query);
        result = response.reset_index();       
        self.data['T2_IFP_CR_p5080'] = result; 

        query = self.statistics['T2_IFP_CR_minmax'];
        response = self.kusto_client.query(query);
        result = response.reset_index();       
        self.data['T2_IFP_CR_minmax'] = result; 
           
        query = self.statistics['T2_CS_CR_p5080'];
        response = self.kusto_client.query(query);
        result = response.reset_index();       
        self.data['T2_CS_CR_p5080'] = result; 

        query = self.statistics['T2_CS_CR_minmax'];
        response = self.kusto_client.query(query);
        result = response.reset_index();       
        self.data['T2_CS_CR_minmax'] = result; 

        query = self.statistics['T2_FS_CR_p5080'];
        response = self.kusto_client.query(query);
        result = response.reset_index();       
        self.data['T2_FS_CR_p5080'] = result; 

        query = self.statistics['T2_FS_CR_minmax'];
        response = self.kusto_client.query(query);
        result = response.reset_index();       
        self.data['T2_FS_CR_minmax'] = result; 

        query = self.statistics['T2_IST_CR_p5080'];
        response = self.kusto_client.query(query);
        result = response.reset_index();       
        self.data['T2_IST_CR_p5080'] = result; 

        query = self.statistics['T2_IST_CR_minmax'];
        response = self.kusto_client.query(query);
        result = response.reset_index();       
        self.data['T2_IST_CR_minmax'] = result; 

        return self.data;

from asyncio.windows_events import NULL
import numpy as np
import os
import random
import pandas as pd
from time import time;

client = DCEPKustoData();
kusto_data = client.query_static_data();
N = 100000
stages = 1

projects = kusto_data['T2_DC_List'];
#print(projects);
ist2cr_project_list = kusto_data['T2_IST2CR_List'];
#print(ist2cr_project_list);
fs2cr_project_list = kusto_data['T2_FS2CR_List'];
#print(fs2cr_project_list);
cs2cr_project_list = kusto_data['T2_CS2CR_List'];  
#print(cs2cr_project_list);
ifp2cr_project_list = kusto_data['T2_IFP2CR_List'];
#print(ifp2cr_project_list);

count = 0;
max_loop = projects.size;
tranche ='T2';

for index, project in projects.iterrows():
  stage_result_file = tranche +"result.xlsx";
  totaltime_result_file = tranche +"totaltime.xlsx";  
  ###Check if the DCShortCode is in IST completed 2 CR list###
  if (ist2cr_project_list["DCShortCode"].str.contains(project["DCShortCode"]).any()):
    stages = 1;
    T =  np.empty(shape=(N,stages + 1));
    TaskTimes=[];
    TotalTime=[];
    Lh=[];
    ist2cr_project = project["DCShortCode"]
    ist2cr_duration = 0
    result = pd.DataFrame
    tasktime = [];
    result_5080 = kusto_data['T2_IST_CR_p5080'];
    result_minmax = kusto_data['T2_IST_CR_minmax'];
    query = client.get_duration_query()['T2_IST_CR']
    query = query.replace("CHI05", ist2cr_project);
    print(query);
    result_ist2cr_duration = client.query_kusto(query);
    ist2cr_duration = result_ist2cr_duration["duration"].values[0];

    p50 = result_5080["percentile_duration_50"].values[0];
    p80 = result_5080["percentile_duration_80"].values[0];
    min = result_minmax["min_duration"].values[0];
    max = result_minmax["max_duration"].values[0];

    tasktime.append(min);
    tasktime.append(p50);
    tasktime.append(max);
    TaskTimes.append(tasktime);
    t0 = time();
    print('Time of start IST to CR loop ', t0);
    for i in range(stages):
      Lh.append((TaskTimes[i][1]-TaskTimes[i][0]) / (TaskTimes[i][2]-TaskTimes[i][0]));

    #Lh running time result
    dt = time()-t0;
    print('Time to run Lh of IST '+ 'dt_loop=%s' % round(dt, 10));
    for p in range(N):
      for i in range(stages):
        trand=random.random()
        if Lh[i] is not pd.NA:
          if (trand < Lh[i]):
            T[p][i] = TaskTimes[i][0] + np.sqrt(trand*(TaskTimes[i][1]-  TaskTimes[i][0])*(TaskTimes[i][2]-TaskTimes[i][0]));
            T[p][i+1] = ist2cr_duration;
          else:
            T[p][i] = TaskTimes[i][2] - np.sqrt((1-trand)*(TaskTimes[i][2] - TaskTimes[i][1]) * (TaskTimes[i][2]-TaskTimes[i][0]));
            T[p][i+1] = ist2cr_duration;
      TotalTime.append(T[p][0]);
    
    Data = pd.DataFrame(T,columns=["MC_Simulation", "DCEP_Duration"]);
    pd.set_option("display.max_columns", None);

    #run simulation loop running time result
    dt_list = time()-t0
    print("Project = ", project["DCShortCode"]);
    print("TotalTime Size = ", len(TotalTime));
    print('Tranche T2'+' Time to run IST to CR Bucket once '+ 'dt_loop=%s' % round(dt_list, 10))
  elif(fs2cr_project_list["DCShortCode"].str.contains(project["DCShortCode"]).any()):
    stage_result_file = "fs2cr_" + stage_result_file;
    totaltime_result_file = "fs2cr_" + totaltime_result_file;
    stages = 1;
    T =  np.empty(shape=(N,stages + 1));
    TaskTimes=[];
    TotalTime=[];
    Lh=[];
    fs2cr_project = project["DCShortCode"];
    fs2cr_duration = 0;
    kusto_query = "";
    result = pd.DataFrame;
    tasktime = [];
    result_5080 = kusto_data['T2_FS_CR_p5080'];
    result_minmax = kusto_data['T2_FS_CR_minmax'];
    query = client.get_duration_query()['T2_FS_CR']
    query = query.replace("CHI05", fs2cr_project);
    result_fs2cr_duration = client.query_kusto(query);
    fs2cr_duration = result_fs2cr_duration["duration"].values[0];

    p50 = result_5080["percentile_duration_50"].values[0];
    p80 = result_5080["percentile_duration_80"].values[0];
    min = result_minmax["min_duration"].values[0];
    max = result_minmax["max_duration"].values[0];
    tasktime.append(min);
    tasktime.append(p50);
    tasktime.append(max);
    TaskTimes.append(tasktime);
                
    #set running time for testing purpose
    t1 = time();
    print('Time of start FS to CR loop ', t1);

    for i in range(stages):
      Lh.append((TaskTimes[i][1]-TaskTimes[i][0]) / (TaskTimes[i][2]-TaskTimes[i][0]));
    #Lh running time result
    dt1 = time()-t1;
    print('Time to run Lh of FS '+ 'dt_loop=%s' % round(dt1, 10));

    for p in range(N):
      for i in range(stages):
        trand=random.random();
        if Lh[i] is not pd.NA:
          if (trand < Lh[i]):
            T[p][i] = TaskTimes[i][0] + np.sqrt(trand*(TaskTimes[i][1]-  TaskTimes[i][0])*(TaskTimes[i][2]-TaskTimes[i][0]));
            T[p][i+1] = fs2cr_duration;
          else:
            T[p][i] = TaskTimes[i][2] - np.sqrt((1-trand)*(TaskTimes[i][2] - TaskTimes[i][1]) * (TaskTimes[i][2]-TaskTimes[i][0]));
            T[p][i+1] = fs2cr_duration;
      TotalTime.append(T[p][0]);
                
      #run simulation loop running time result
      #dt1_loop = time()-t1
      #print(fs2cr_project+' in Tranche '+tranche+' Time to run simulation loop '+ 'dt1_loop=%s' % round(dt1_loop, 10))

    Data = pd.DataFrame(T,columns=["FS to CR", "DCEP FS to CR duration"]);
    pd.set_option("display.max_columns", None);
            
    #run simulation loop running time result
    dt1_list = time()-t1;
    print("Project = ", project["DCShortCode"]);
    print("TotalTime Size = ", len(TotalTime));
    print('Tranche '+tranche+' Time to run FS to CR Bucket once '+ 'dt1_list=%s' % round(dt1_list, 10));

    #append_to_xlsx(Data.describe(), stage_result_file, fs2cr_project)
    #append_to_xlsx(pd.DataFrame(TotalTime), totaltime_result_file, fs2cr_project)
  elif(cs2cr_project_list["DCShortCode"].str.contains(project["DCShortCode"]).any()):
    stage_result_file = "cs2cr_" + stage_result_file;
    totaltime_result_file = "cs2cr_" + totaltime_result_file;
    stages = 1;
    T =  np.empty(shape=(N,stages + 1));
    TaskTimes=[];
    TotalTime=[];
    Lh=[];
    cs2cr_project = project["DCShortCode"];
    cs2cr_duration = 0;
    kusto_query = "";
    result = pd.DataFrame;
    tasktime = [];
    result_5080 = kusto_data['T2_CS_CR_p5080'];
    result_minmax = kusto_data['T2_CS_CR_minmax'];

    query = client.get_duration_query()['T2_CS_CR'];
    query = query.replace("CHI05", cs2cr_project);
    print(query);
    result_cs2cr_duration = client.query_kusto(query);
    cs2cr_duration = result_cs2cr_duration["duration"].values[0];
    
    p50 = result_5080["percentile_duration_50"].values[0];
    p80 = result_5080["percentile_duration_80"].values[0];
    min = result_minmax["min_duration"].values[0];
    max = result_minmax["max_duration"].values[0];
    tasktime.append(min);
    tasktime.append(p50);
    tasktime.append(max);
    TaskTimes.append(tasktime);
    
    #set running time for testing purpose
    t2 = time();
    print('Time of start CS to CR loop ', t2);

    for i in range(stages):
      Lh.append((TaskTimes[i][1]-TaskTimes[i][0]) / (TaskTimes[i][2]-TaskTimes[i][0]))
    #Lh running time result
    dt2 = time()-t2;
    print('Time to run Lh of FS '+ 'dt2=%s' % round(dt2, 10));

    for p in range(N):
      for i in range(stages):
        trand=random.random();
        if Lh[i] is not pd.NA:
          if (trand < Lh[i]):
            T[p][i] = TaskTimes[i][0] + np.sqrt(trand*(TaskTimes[i][1]-  TaskTimes[i][0])*(TaskTimes[i][2]-TaskTimes[i][0]));
            T[p][i+1] = cs2cr_duration;
          else:
            T[p][i] = TaskTimes[i][2] - np.sqrt((1-trand)*(TaskTimes[i][2] - TaskTimes[i][1]) * (TaskTimes[i][2]-TaskTimes[i][0]));
            T[p][i+1] = cs2cr_duration;

      TotalTime.append(T[p][0]);
      #run simulation loop running time result
      #dt2_loop = time()-t2
      #print(cs2cr_project+' in Tranche '+tranche+' Time to run simulation loop '+ 'dt2_loop=%s' % round(dt2_loop, 10))
                

    Data = pd.DataFrame(T,columns=["CS to CR", "DCEP CS to CR duration"])
    pd.set_option("display.max_columns", None);
            
    #run simulation loop running time result
    dt2_list = time()-t2;
    print("Project = ", project["DCShortCode"]);
    print('Tranche '+tranche+' Time to run CS to CR Bucket once '+ 'dt2_list=%s' % round(dt2_list, 10)) 

    #append_to_xlsx(Data.describe(), stage_result_file, cs2cr_project)
    #append_to_xlsx(pd.DataFrame(TotalTime), totaltime_result_file, cs2cr_project)
  elif(ifp2cr_project_list["DCShortCode"].str.contains(project["DCShortCode"]).any()):
    stage_result_file = "ifp2cr_" + stage_result_file;
    totaltime_result_file = "ifp2cr_" + totaltime_result_file;
    stages = 1;
    T =  np.empty(shape=(N,stages + 1));
    TaskTimes=[];
    TotalTime=[];
    Lh=[];
    ifp2cr_project = project["DCShortCode"];
    ifp2cr_duration = 0;
    kusto_query = "";
    result = pd.DataFrame;
    tasktime = [];
    result_5080 = kusto_data['T2_IFP_CR_p5080'];
    result_minmax = kusto_data['T2_IFP_CR_minmax'];

    query = client.get_duration_query()['T2_IFP_CR'];
    query = query.replace("CHI05", ifp2cr_project);
    print(query);
    result_ifp2cr_duration = client.query_kusto(query);
    ifp2cr_duration = result_ifp2cr_duration["duration"].values[0];

    p50 = result_5080["percentile_duration_50"].values[0];
    p80 = result_5080["percentile_duration_80"].values[0];
    min = result_minmax["min_duration"].values[0];
    max = result_minmax["max_duration"].values[0];
    tasktime.append(min);
    tasktime.append(p50);
    tasktime.append(max);
    TaskTimes.append(tasktime);

            
    #set running time for testing purpose
    t3 = time()
    print('Time of start IFP to CR loop ', t3)

    for i in range(stages):
      Lh.append((TaskTimes[i][1]-TaskTimes[i][0]) / (TaskTimes[i][2]-TaskTimes[i][0]))
      #Lh running time result
    dt3 = time()-t3
    print('Time to run Lh of IFP '+ 'dt3=%s' % round(dt3, 10))

    for p in range(N):
      for i in range(stages):
        trand=random.random()
        if Lh[i] is not pd.NA:
          if (trand < Lh[i]):
            T[p][i] = TaskTimes[i][0] + np.sqrt(trand*(TaskTimes[i][1]-  TaskTimes[i][0])*(TaskTimes[i][2]-TaskTimes[i][0]));
            T[p][i+1] = ifp2cr_duration;
          else:
            T[p][i] = TaskTimes[i][2] - np.sqrt((1-trand)*(TaskTimes[i][2] - TaskTimes[i][1]) * (TaskTimes[i][2]-TaskTimes[i][0]));
            T[p][i+1] = ifp2cr_duration;

      TotalTime.append(T[p][0]);
                
    #run simulation loop running time result
    #dt3_loop = time()-t3
    #print(ifp2cr_project+' in Tranche '+tranche+' Time to run simulation loop '+ 'dt3_loop=%s' % round(dt3_loop, 10))

    Data = pd.DataFrame(T,columns=["IFP to CR", "DCEP IFP to CR duration"])
    pd.set_option("display.max_columns", None)

    #run simulation loop running time result
    dt3_list = time()-t3
    print("Project = ", project["DCShortCode"]);
    print("TotalTime Size = ", len(TotalTime));
    print('Tranche '+tranche+' Time to run IFP to CR Bucket once '+ 'dt3_loop=%s' % round(dt3_list, 10))              

    #append_to_xlsx(Data.describe(), stage_result_file, ifp2cr_project)
    #append_to_xlsx(pd.DataFrame(TotalTime), totaltime_result_file, ifp2cr_project)

  count = index;
  if count == max_loop:
    break



Time of start FS to CR loop  1706654217.894035
Time to run Lh of FS dt_loop=0.0009958744
Project =  AKL02
TotalTime Size =  100000
Tranche T2 Time to run FS to CR Bucket once dt1_list=0.5011472702
Time of start FS to CR loop  1706654219.122046
Time to run Lh of FS dt_loop=0.0
Project =  AMS13
TotalTime Size =  100000
Tranche T2 Time to run FS to CR Bucket once dt1_list=0.4569950104
Time of start FS to CR loop  1706654220.259537
Time to run Lh of FS dt_loop=0.0
Project =  AMS14
TotalTime Size =  100000
Tranche T2 Time to run FS to CR Bucket once dt1_list=0.4789898396

        let CSFullSet = 
        (
          DCEPAllMilestonesCurrent
          | where ProjectType in ("Build","Build PH","OAI Build") 
          | summarize by ProjectName, Tranche
          | join kind = leftouter  
          (
            DCEPAllMilestonesCurrent 
            | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build")
            | summarize by DCShortCode, ProjectName,Tranche, L2Milestone